In [1]:
# set up
from collections import defaultdict
import numpy as np
import pandas as pd
import requests
from scipy import stats
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import metrics

from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import f1_score

import json

Using TensorFlow backend.


In [2]:
class Intent_classifiction(object):
    def __init__(self):
        self.url_data = "https://github.com/google-research-datasets/Taskmaster/raw/master/TM-1-2019/self-dialogs.json"
        self.data = requests.get(self.url_data).json()
        
        # Phrases to intents
        self.phrs2intents = {}
        
        # Phrases embedded
        self.phrs2vec = {}
        
        # phrases lists
        self.X = None
        
        # Word Embedding models loading
        use_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
        self.use = hub.load(use_url)
        print("module use loaded")
        self.elmo = hub.load("https://tfhub.dev/google/elmo/3")
        print("module elmo loaded")

    def ELMO(self, x):
        return self.elmo.signatures['default'](tf.convert_to_tensor(x))

    def USE(self, x):
        return self.use(x)

    def intents_clustering(self):
        """ Establishing a dictionary from phrases to intents """
        self.phrs2intents = {}
        number_of_other = 10000;
        for i in range(len(self.data)):
            for ut in self.data[i]['utterances']:
                if ut['speaker'] == 'USER':
                    if 'segments' in ut.keys():
                        for seg in ut['segments']:
                            if 'annotations' in seg.keys():
                                for anno in seg['annotations']:
                                    name = anno['name']
                                    if ut['text'] not in self.phrs2intents.keys():
                                        self.phrs2intents[ ut['text'] ] = [name]
                                    elif name not in self.phrs2intents[ ut['text'] ]:
                                        self.phrs2intents[ ut['text'] ].append(name)
                    else:
                        if number_of_other > 0:
                            self.phrs2intents[ ut['text'] ] = ['other']
                            number_of_other -= 1
        self.X = np.array(list(self.phrs2intents.keys()))
        
    def word_embedding(self):
        self.intents_clustering()
        print("intents clustering completed!")
        if not os.path.exists("phr_embedded.json"):
            self.phrs2vec = {}
            
            for i in range(len(self.X)):
                if i % 1000 == 0:
                    print("Embedding Progress: ", i / len(self.X))
                self.phrs2vec[ self.X[i] ]  = str( list( np.array(self.ELMO([self.X[i]])['default'][0]) ))
            with open('phr_embedded.json', 'w') as fp:
                json.dump(self.phrs2vec, fp)
                           
        with open('phr_embedded.json', 'r') as fp:
            self.phrs2vec = json.load(fp)
        print("embedded data loading completed")              
        
        self.phrs = list(self.phrs2vec.keys())
        for i in range( len(self.phrs) ):
            if i % 1000 == 0:
                    print("Embedding Progress: ", i / len(self.X))
            self.phrs2vec[self.phrs[i]] = np.array( eval( self.phrs2vec[self.phrs[i]]) )
    
    def get_embedded_data_and_clustered_data(self):
        return self.phrs2intents, self.phrs2vec

In [3]:
IC = Intent_classifiction()

module use loaded
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


module elmo loaded


In [4]:
IC.intents_clustering()

In [5]:
len( IC.phrs2intents.keys() )

41824

In [6]:
IC.phrs2intents

{"Hi, I'm looking to book a table for Korean fod.": ['other'],
 'Somewhere in Southern NYC, maybe the East Village?': ['restaurant_reservation.location.restaurant.accept'],
 "That's great. So I need a table for tonight at 7 pm for 8 people. We don't want to sit at the bar, but anywhere else is fine.": ['restaurant_reservation.type.seating',
  'restaurant_reservation.time.reservation',
  'restaurant_reservation.num.guests'],
 'What times are available?': ['other'],
 "Yikes, we can't do those times.": ['other'],
 'Let me check.': ['other'],
 'Lets try Boka, are they free for 8 people at 7?': ['restaurant_reservation.name.restaurant.accept',
  'restaurant_reservation.num.guests.accept',
  'restaurant_reservation.time.reservation.accept'],
 "Great, let's book that.": ['other'],
 "No, that's it, just book.": ['other'],
 'Yes please.': ['other',
  'coffee_ordering.preference.accept',
  'coffee_ordering.preference'],
 'Hi I would like to see if the Movie What Men Want is playing here.': ['mov

In [7]:
IC.word_embedding()

intents clustering completed!
Embedding Progress:  0.0
Embedding Progress:  0.023909716908951797
Embedding Progress:  0.047819433817903594
Embedding Progress:  0.0717291507268554
Embedding Progress:  0.09563886763580719
Embedding Progress:  0.11954858454475899
Embedding Progress:  0.1434583014537108
Embedding Progress:  0.16736801836266257
Embedding Progress:  0.19127773527161437
Embedding Progress:  0.21518745218056617
Embedding Progress:  0.23909716908951797
Embedding Progress:  0.2630068859984698
Embedding Progress:  0.2869166029074216
Embedding Progress:  0.3108263198163734
Embedding Progress:  0.33473603672532515
Embedding Progress:  0.358645753634277
Embedding Progress:  0.38255547054322875
Embedding Progress:  0.4064651874521806
Embedding Progress:  0.43037490436113235
Embedding Progress:  0.4542846212700842
Embedding Progress:  0.47819433817903595
Embedding Progress:  0.5021040550879877
Embedding Progress:  0.5260137719969395
Embedding Progress:  0.5499234889058914
Embedding Pr

In [15]:
np.random.randint(100)

71

In [28]:
class Intent_detection(object):
    ''' In this class , we use the notations below'''
    ''' topic means: the grand theme of the conversation, e.g.: "restaurant" '''
    ''' subtopic means: the small theme belonging to the topic, e.g. "restaurant.time" '''
    ''' intents means: the intent of the user, which kind of express the attitude of user, e.g. "restaurant.time.accept" '''

    def __init__(self, p2i, p2v):
        self.p2i = p2i
        self.p2v = p2v
        self.topic = ['auto', 'coffee', 'movie', 'pizza', 'restaurant', 'uber', 'other']
        self.phrs_zoo = np.array(list(p2i.keys()))
        self.topic2phrs_zoo = {'auto':[], 'coffee':[], 'movie':[], 'pizza':[], 'restaurant':[], 'uber':[], 'other':[]}
        
        # Load word embedding model
        self.elmo = hub.load("https://tfhub.dev/google/elmo/3")
        print("module elmo loaded")
        
        # Load the phrases zoo
        for t in self.topic:
            for i in range(len(self.phrs_zoo)):
                if self.p2i[ self.phrs_zoo[i] ][0].find(t) > -1:
                    self.topic2phrs_zoo[ t ].append(self.phrs_zoo[i])
     
        #  the sub_topic dictionary stores the subtopics of each topic and their coding
        self.topic2sub_topic = {'restaurant': {"time": 1, "location": 2, "num": 3, "name": 4, "type": 5},
                          'auto': {"name": 1, "year": 2, "reason": 3, "date": 4},
                          'movie': {"time": 1, "location": 2, "num": 3, "name": 4, "type": 5, "price": 6},
                          'coffee': {"size": 1, "location": 2, "num": 3, "name": 4, "type": 5, "preference": 6},
                          'pizza': {"size": 1, "location": 2, "preference": 3, "name": 4, "type": 5},
                          'uber': {"time": 1, "location": 2, "num": 3, "duration": 4, "type": 5},
                          'other':{"other":1}  }
        
        # load phrases to each subtopic
        self.phrs2sub_topic = {'auto':{}, 'coffee':{}, 'movie':{}, 'pizza':{}, 'restaurant':{}, 'uber':{},  'other':{} }
        
        for t in self.topic:
            if t != 'other':
                sub_topic = list( self.topic2sub_topic[t].keys() )
                for p in self.topic2phrs_zoo[t]:
                    sub_topic_of_p =  len(sub_topic) * [ 0 ]
                    for i in self.p2i[p]:
                        for st in sub_topic:
                            if i.find(st) > -1:
                                sub_topic_of_p[ self.topic2sub_topic[t][st] - 1 ] = 1
                    self.phrs2sub_topic[t][p] = tuple(sub_topic_of_p)
        
        
        #       Two dictionarys to stock the training set according to topics
        self.model_zoo = {'auto':{}, 'coffee':{}, 'movie':{}, 'pizza':{}, 'restaurant':{}, 'uber':{}};
        
    
    def my_train_test_split(self, X, y, test_size=0.2):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
        return X_train, X_test, y_train, y_test
    
    def f1_score_model(self, model, X, y):
        """Get the F1 score of a specific model on a specific data (X, y)"""

        prediction = model.predict_classes(X)
        f1_macro = f1_score(y, prediction, average='macro')
        f1_micro = f1_score(y, prediction, average='macro')
        print("f1_macro: ", f1_score(y, prediction, average='macro'))
        print("f1_micro: ", f1_score(y, prediction, average="micro"))
        print("f1_weighted: ", f1_score(y, prediction, average="weighted"))
        return f1_macro, f1_micro
    
    def get_data(self, t, st, total_data_number = 10000):
        """ Get the data for training and validation for specific """
        
        X = []
        y = []
        num_pos = 0
        num_neg = 0
        num_other = 0
        for p in self.topic2phrs_zoo[t]:
            if num_pos > total_data_number // 2 and num_neg > total_data_number // 4:
                break
            if self.phrs2sub_topic[t][p][  self.topic2sub_topic[t][st] - 1  ] > 0:
                if num_pos >= -0:
                    X.append( p2v[p] )
                    y.append( 1 )
                    num_pos += 1
            else:
                if num_neg <= num_pos // 2:
                    X.append( p2v[p] )
                    y.append( 0 )
                    num_neg += 1
        labeled_sample = len(y)
        for i in range(len(y) // 3):
            p = self.topic2phrs_zoo['other'][np.random.randint(len( ID.topic2phrs_zoo['other'] ))]
            X.append(p2v[p])
            y.append(0)
            num_other += 1;
        print("We get: ")
        print(num_pos, " postive sample")
        print(num_neg, " sample of wrong intents")
        print(num_other, " sample of no intents")
        
        return np.array(X), np.array(y)
    
    def train_model(self):
        """ Training all the subtopic and topics"""
        self.best_epoch = {'auto':{}, 'coffee':{}, 'movie':{}, 'pizza':{}, 'restaurant':{}, 'uber':{} }
        self.best_f1 = {'auto':{}, 'coffee':{}, 'movie':{}, 'pizza':{}, 'restaurant':{}, 'uber':{} }
        for t in self.topic:
            if t != 'other':
                for st in self.topic2sub_topic[t].keys():

                    print("Now training the classsfier for topic: ", t, " ; intent: ", st)
                    print(128 * "=")
                    print("Input: str; Output: boolean(if the str contents the intent: ", st, " ).")
                    print(64 * "-")
                    X, y = self.get_data(t, st)
                    print("data_loaded!")
                    X_train, X_dev, y_train, y_dev = self.my_train_test_split(X, y)
                    best_f1 = 0
                    for e in range(1,10):
                        model = tf.keras.Sequential()
                        model.add(tf.keras.layers.InputLayer(input_shape=[1024, ]))
                        model.add(tf.keras.layers.Dense(64, activation='relu'))
                        model.add(tf.keras.layers.Dense(64, activation='relu'))
                        model.add(tf.keras.layers.Dense(1, activation='relu'))
                        model.compile(loss='mean_squared_logarithmic_error', optimizer='adam', metrics=[metrics.mae, metrics.categorical_accuracy])
                        model.fit(X_train, y_train, epochs=e, batch_size=128)
                        print("f1_score on dev set: ")
                        f1 = self.f1_score_model(model, X_dev, y_dev)[0]
                        if f1 > best_f1:
                            self.model_zoo[t][st] = model
                            model.save_weights("intent_detection_model/%s/%s.h5" %(t,st))
                            self.best_epoch[t][st] = e
                            self.best_f1[t][st] = f1
                            best_f1 = f1

                        print(64*"=")
                    print()
    
    def load_model(self):
        """Load all the models, if model does not exist, retrain it and store it"""
        for t in self.topic:
            if t != "other":
                print("Loading models of topic: ", t)
                for st in self.topic2sub_topic[t].keys():
                    model = tf.keras.Sequential()
                    model.add(tf.keras.layers.InputLayer(input_shape=[1024, ]))
                    model.add(tf.keras.layers.Dense(64, activation='relu'))
                    model.add(tf.keras.layers.Dense(64, activation='relu'))
                    model.add(tf.keras.layers.Dense(1, activation='relu'))
                    model.compile(loss='mean_squared_logarithmic_error', optimizer='adam', metrics=[metrics.mae, metrics.categorical_accuracy])

                    if not os.path.exists("intent_detection_model/%s/%s.h5" %(t,st)):
                        print("Now training the classsfier for topic: ", t, " ; intent: ", st)
                        print(64 * "=")
                        X, y = self.get_data(t, st)
                        print("data_loaded!")
                        X_train, X_dev, y_train, y_dev = self.my_train_test_split(X, y)
                        model.fit(X_train, y_train, epochs=3, batch_size=128)
                        model.save_weights("intent_detection_model/%s/%s.h5" %(t,st))
                        print("f1_score on dev set: ")
                        self.f1_score_model(model, X_dev, y_dev)
                        print(64*"=")
                        print()
                    else:
                        model.load_weights("intent_detection_model/%s/%s.h5" %(t,st))
                    self.model_zoo[t][st] = model
                
    def ELMO(self, x):
        return self.elmo.signatures['default'](tf.convert_to_tensor(x))
    
    def predict(self, phr, t):
        '''Enter one phrase and get the prediction of if this phrase contains each intent'''
        '''phr: str, t: str'''
        
        X = np.array(self.ELMO([phr]) ['default'])
            
        st_array = list( self.topic2sub_topic[t].keys() )
        res = []
        for st in st_array:
            res.append( self.model_zoo[t][st].predict(X)[0][0] )
        
        return res


In [8]:
p2i, p2v = IC.get_embedded_data_and_clustered_data()

In [12]:
len( p2i.keys() )

41824

In [11]:
len( p2v.keys() )

41824

In [29]:
ID = Intent_detection(p2i, p2v)

module elmo loaded


In [30]:
len( ID.topic2phrs_zoo['other'] )

7475

In [31]:
ID.get_data('restaurant', 'time')

We get: 
2257  postive sample
1129  sample of wrong intents
1128  sample of no intents


(array([[ 0.47903794,  0.19902727,  0.09933247, ..., -0.11180887,
          0.04152172, -0.06517152],
        [ 0.07155436, -0.2274629 , -0.08247091, ..., -0.27136704,
          0.2191311 ,  0.09496994],
        [ 0.21138269, -0.4100072 ,  0.2637415 , ..., -0.06332628,
         -0.08134664, -0.09446161],
        ...,
        [-0.05204089,  0.00144664, -0.14870936, ..., -0.19089068,
          0.2146221 ,  0.14373568],
        [-0.10554998,  0.05981535, -0.00876214, ..., -0.19736335,
          0.25938687,  0.11582933],
        [ 0.07074019, -0.43728518,  0.08391234, ..., -0.6154096 ,
         -0.06384839,  0.08967131]]), array([0, 1, 1, ..., 0, 0, 0]))

In [32]:
ID.train_model()

Now training the classsfier for topic:  auto  ; intent:  name
Input: str; Output: boolean(if the str contents the intent:  name  ).
----------------------------------------------------------------
We get: 
2616  postive sample
1309  sample of wrong intents
1308  sample of no intents
data_loaded!
Train on 4186 samples
4186/4186 [==============================] - 1s 176us/sample - loss: 0.0492 - mean_absolute_error: 0.2109 - categorical_accuracy: 1.0000
f1_score on dev set: 
f1_macro:  0.9531091049846769
f1_micro:  0.9531996179560649
f1_weighted:  0.9531661675101172
Train on 4186 samples
Epoch 1/2
4186/4186 [==============================] - 1s 244us/sample - loss: 0.0517 - mean_absolute_error: 0.2233 - categorical_accuracy: 1.0000
Epoch 2/2
4186/4186 [==============================] - 0s 43us/sample - loss: 0.0137 - mean_absolute_error: 0.0970 - categorical_accuracy: 1.0000
f1_score on dev set: 
f1_macro:  0.9732498722534492
f1_micro:  0.9732569245463228
f1_weighted:  0.9732619026354101

Epoch 7/9
4186/4186 [==============================] - 0s 46us/sample - loss: 0.0044 - mean_absolute_error: 0.0447 - categorical_accuracy: 1.0000
Epoch 8/9
4186/4186 [==============================] - 0s 45us/sample - loss: 0.0037 - mean_absolute_error: 0.0379 - categorical_accuracy: 1.0000
Epoch 9/9
4186/4186 [==============================] - 0s 45us/sample - loss: 0.0032 - mean_absolute_error: 0.0341 - categorical_accuracy: 1.0000
f1_score on dev set: 
f1_macro:  0.977048701180081
f1_micro:  0.9770773638968482
f1_weighted:  0.9770711665526822

Now training the classsfier for topic:  auto  ; intent:  year
Input: str; Output: boolean(if the str contents the intent:  year  ).
----------------------------------------------------------------
We get: 
964  postive sample
483  sample of wrong intents
482  sample of no intents
data_loaded!
Train on 1543 samples
1543/1543 [==============================] - 1s 401us/sample - loss: 0.0607 - mean_absolute_error: 0.2478 - categorical_accuracy: 1

1543/1543 [==============================] - 0s 52us/sample - loss: 0.0130 - mean_absolute_error: 0.1070 - categorical_accuracy: 1.0000
Epoch 3/9
1543/1543 [==============================] - 0s 49us/sample - loss: 0.0066 - mean_absolute_error: 0.0686 - categorical_accuracy: 1.0000
Epoch 4/9
1543/1543 [==============================] - 0s 47us/sample - loss: 0.0047 - mean_absolute_error: 0.0559 - categorical_accuracy: 1.0000
Epoch 5/9
1543/1543 [==============================] - 0s 51us/sample - loss: 0.0035 - mean_absolute_error: 0.0509 - categorical_accuracy: 1.0000
Epoch 6/9
1543/1543 [==============================] - 0s 50us/sample - loss: 0.0029 - mean_absolute_error: 0.0448 - categorical_accuracy: 1.0000
Epoch 7/9
1543/1543 [==============================] - 0s 53us/sample - loss: 0.0022 - mean_absolute_error: 0.0412 - categorical_accuracy: 1.0000
Epoch 8/9
1543/1543 [==============================] - 0s 56us/sample - loss: 0.0018 - mean_absolute_error: 0.0367 - categorical_accur

2588/2588 [==============================] - 0s 47us/sample - loss: 0.0068 - mean_absolute_error: 0.0618 - categorical_accuracy: 1.0000
Epoch 8/8
2588/2588 [==============================] - 0s 47us/sample - loss: 0.0056 - mean_absolute_error: 0.0562 - categorical_accuracy: 1.0000
f1_score on dev set: 
f1_macro:  0.9644993247154158
f1_micro:  0.9645061728395061
f1_weighted:  0.9644978029100624
Train on 2588 samples
Epoch 1/9
2588/2588 [==============================] - 1s 243us/sample - loss: 0.0607 - mean_absolute_error: 0.2642 - categorical_accuracy: 1.0000
Epoch 2/9
2588/2588 [==============================] - 0s 45us/sample - loss: 0.0248 - mean_absolute_error: 0.1510 - categorical_accuracy: 1.0000
Epoch 3/9
2588/2588 [==============================] - 0s 44us/sample - loss: 0.0187 - mean_absolute_error: 0.1204 - categorical_accuracy: 1.0000
Epoch 4/9
2588/2588 [==============================] - 0s 48us/sample - loss: 0.0146 - mean_absolute_error: 0.1014 - categorical_accuracy: 1.0

1530/1530 [==============================] - 0s 61us/sample - loss: 0.0420 - mean_absolute_error: 0.2086 - categorical_accuracy: 1.0000
Epoch 3/8
1530/1530 [==============================] - 0s 59us/sample - loss: 0.0305 - mean_absolute_error: 0.1762 - categorical_accuracy: 1.0000
Epoch 4/8
1530/1530 [==============================] - 0s 57us/sample - loss: 0.0231 - mean_absolute_error: 0.1441 - categorical_accuracy: 1.0000
Epoch 5/8
1530/1530 [==============================] - 0s 61us/sample - loss: 0.0179 - mean_absolute_error: 0.1217 - categorical_accuracy: 1.0000
Epoch 6/8
1530/1530 [==============================] - 0s 61us/sample - loss: 0.0141 - mean_absolute_error: 0.1055 - categorical_accuracy: 1.0000
Epoch 7/8
1530/1530 [==============================] - 0s 64us/sample - loss: 0.0115 - mean_absolute_error: 0.0932 - categorical_accuracy: 1.0000
Epoch 8/8
1530/1530 [==============================] - 0s 62us/sample - loss: 0.0092 - mean_absolute_error: 0.0824 - categorical_accur

...

3374/3374 [==============================] - 0s 40us/sample - loss: 0.0031 - mean_absolute_error: 0.0409 - categorical_accuracy: 1.0000
f1_score on dev set: 
f1_macro:  0.9691770046745776
f1_micro:  0.9691943127962085
f1_weighted:  0.9691943127962085

Now training the classsfier for topic:  uber  ; intent:  num
Input: str; Output: boolean(if the str contents the intent:  num  ).
----------------------------------------------------------------
We get: 
343  postive sample
172  sample of wrong intents
171  sample of no intents
data_loaded!
Train on 548 samples
548/548 [==============================] - 1s 1ms/sample - loss: 0.1906 - mean_absolute_error: 0.4848 - categorical_accuracy: 1.0000
f1_score on dev set: 
f1_macro:  0.5411602843384544
f1_micro:  0.5797101449275363
f1_weighted:  0.5334503122206381
Train on 548 samples
Epoch 1/2
548/548 [==============================] - 1s 1ms/sample - loss: 0.1795 - mean_absolute_error: 0.4766 - categorical_accuracy: 1.0000
Epoch 2/2
548/548 [====

548/548 [==============================] - 0s 51us/sample - loss: 0.0168 - mean_absolute_error: 0.1315 - categorical_accuracy: 1.0000
Epoch 6/9
548/548 [==============================] - 0s 53us/sample - loss: 0.0125 - mean_absolute_error: 0.1130 - categorical_accuracy: 1.0000
Epoch 7/9
548/548 [==============================] - 0s 51us/sample - loss: 0.0092 - mean_absolute_error: 0.0970 - categorical_accuracy: 1.0000
Epoch 8/9
548/548 [==============================] - 0s 55us/sample - loss: 0.0076 - mean_absolute_error: 0.0925 - categorical_accuracy: 1.0000
Epoch 9/9
548/548 [==============================] - 0s 49us/sample - loss: 0.0054 - mean_absolute_error: 0.0761 - categorical_accuracy: 1.0000
f1_score on dev set: 
f1_macro:  0.8908400569530137
f1_micro:  0.8913043478260869
f1_weighted:  0.891252759951301

Now training the classsfier for topic:  uber  ; intent:  duration
Input: str; Output: boolean(if the str contents the intent:  duration  ).
-----------------------------------

Epoch 3/9
32/32 [==============================] - 0s 281us/sample - loss: 0.2102 - mean_absolute_error: 0.4375 - categorical_accuracy: 1.0000
Epoch 4/9
32/32 [==============================] - 0s 219us/sample - loss: 0.2102 - mean_absolute_error: 0.4375 - categorical_accuracy: 1.0000
Epoch 5/9
32/32 [==============================] - 0s 219us/sample - loss: 0.2102 - mean_absolute_error: 0.4375 - categorical_accuracy: 1.0000
Epoch 6/9
32/32 [==============================] - 0s 218us/sample - loss: 0.2102 - mean_absolute_error: 0.4375 - categorical_accuracy: 1.0000
Epoch 7/9
32/32 [==============================] - 0s 249us/sample - loss: 0.2102 - mean_absolute_error: 0.4375 - categorical_accuracy: 1.0000
Epoch 8/9
32/32 [==============================] - 0s 218us/sample - loss: 0.2102 - mean_absolute_error: 0.4375 - categorical_accuracy: 1.0000
Epoch 9/9
32/32 [==============================] - 0s 219us/sample - loss: 0.2102 - mean_absolute_error: 0.4375 - categorical_accuracy: 1.0000

2630/2630 [==============================] - 0s 43us/sample - loss: 0.0118 - mean_absolute_error: 0.0915 - categorical_accuracy: 1.0000
Epoch 8/8
2630/2630 [==============================] - 0s 42us/sample - loss: 0.0103 - mean_absolute_error: 0.0862 - categorical_accuracy: 1.0000
f1_score on dev set: 
f1_macro:  0.9178572419341415
f1_micro:  0.9179331306990881
f1_weighted:  0.9179634862050668
Train on 2630 samples
Epoch 1/9
2630/2630 [==============================] - 1s 266us/sample - loss: 0.0739 - mean_absolute_error: 0.3001 - categorical_accuracy: 1.0000
Epoch 2/9
2630/2630 [==============================] - 0s 49us/sample - loss: 0.0363 - mean_absolute_error: 0.1902 - categorical_accuracy: 1.0000
Epoch 3/9
2630/2630 [==============================] - 0s 42us/sample - loss: 0.0282 - mean_absolute_error: 0.1603 - categorical_accuracy: 1.0000
Epoch 4/9
2630/2630 [==============================] - 0s 41us/sample - loss: 0.0237 - mean_absolute_error: 0.1435 - categorical_accuracy: 1.0

KeyError: "Hi, I'm looking to book a table for Korean fod."

In [60]:
ID.best_epoch

{'auto': {'name': 3, 'year': 5, 'reason': 4, 'date': 8},
 'coffee': {'size': 5,
  'location': 3,
  'num': 7,
  'name': 4,
  'type': 7,
  'preference': 6},
 'movie': {'time': 4,
  'location': 6,
  'num': 6,
  'name': 6,
  'type': 8,
  'price': 5},
 'pizza': {'size': 5, 'location': 4, 'preference': 4, 'name': 9, 'type': 8},
 'restaurant': {'time': 5, 'location': 5, 'num': 9, 'name': 9, 'type': 6},
 'uber': {'time': 9, 'location': 3, 'num': 9, 'duration': 9, 'type': 4}}

In [33]:
ID.best_epoch

{'auto': {'name': 8, 'year': 8, 'reason': 6, 'date': 8},
 'coffee': {'size': 5,
  'location': 5,
  'num': 6,
  'name': 9,
  'type': 9,
  'preference': 7},
 'movie': {'time': 8,
  'location': 8,
  'num': 5,
  'name': 6,
  'type': 7,
  'price': 4},
 'pizza': {'size': 6, 'location': 7, 'preference': 5, 'name': 7, 'type': 7},
 'restaurant': {'time': 9, 'location': 6, 'num': 6, 'name': 4, 'type': 7},
 'uber': {'time': 9, 'location': 5, 'num': 9, 'duration': 5, 'type': 3}}

In [34]:
ID.best_f1

{'auto': {'name': 0.9808810424670942,
  'year': 0.9896303460133248,
  'reason': 0.9675918979744935,
  'date': 0.9554922106241754},
 'coffee': {'size': 0.9412760550101453,
  'location': 0.9779668607254814,
  'num': 0.7460423634336678,
  'name': 0.927499546872168,
  'type': 0.949705160440613,
  'preference': 0.8765083602863972},
 'movie': {'time': 0.9580514436200531,
  'location': 0.9484513274336284,
  'num': 0.9682597200413362,
  'name': 0.9236032388663968,
  'type': 0.9431030815899348,
  'price': 0.8875000000000001},
 'pizza': {'size': 0.9547611763946462,
  'location': 0.9402645427392975,
  'preference': 0.8896806019233643,
  'name': 0.8814423189423188,
  'type': 0.9397979708344426},
 'restaurant': {'time': 0.9612334639657043,
  'location': 0.9598554797270173,
  'num': 0.9646907118746919,
  'name': 0.9241467453108507,
  'type': 0.93424938100148},
 'uber': {'time': 0.9366682700016034,
  'location': 0.9774790147279604,
  'num': 0.8908400569530137,
  'duration': 0.8831168831168831,
  'typ

In [40]:
test_phrases = np.array([["I want some newest movie in the center city!"], ["This Evening"],
                  ["We have totally four people"], ["My dad, mom, me and my sister"], ["I would like some Thai food"],
                  ["Mc Donald is great"], ["Thank you"]])

In [41]:
t = 'restaurant'
for phr in test_phrases:
    intent = []
    print(phr[0])
    res = ID.predict( phr[0] , t  )
    sub_topic_array = list( ID.topic2sub_topic[t].keys() )
    for st in sub_topic_array:
        if res[ ID.topic2sub_topic[t][st] - 1] > 0.5:
            intent.append(st)
    if len(intent) > 0:
        print("Detected intents: ", end="")
        for i in intent:
            print(i, end=" ")
            
        print("\n" + 32*"-=")

I want some newest movie in the center city!
Detected intents: location 
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
This Evening
Detected intents: time 
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
We have totally four people
Detected intents: num 
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
My dad, mom, me and my sister
Detected intents: num 
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
I would like some Thai food
Mc Donald is great
Detected intents: name 
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Thank you
